In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet import ResNet18
from keras.applications.resnet import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
train_dir = r'chest_xray\train'
test_dir = r'chest_xray\test'
val_dir = r'chest_xray\val'

In [ ]:
IMAGE_SIZE = [224, 224]

In [ ]:
resnet = ResNet18(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
folders = glob('chest_xray/train/*')
print(folders)

In [ ]:
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

In [ ]:
model = Model(inputs=resnet.input, outputs=prediction)
model.summary()

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(
    'chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
test_set = test_datagen.flow_from_directory(
    'chest_xray/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
val_set = val_datagen.flow_from_directory(
    'chest_xray/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
early_stopping = EarlyStopping(
    min_delta=0.001,  # minimum amount of change to count as an improvement
    patience=20,  # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [ ]:
r = model.fit(
    training_set,
    validation_data=val_set,
    epochs=500,
    steps_per_epoch=len(training_set),
    validation_steps=len(val_set),
    callbacks=[early_stopping]
)

In [ ]:
metrics_data = []

In [ ]:
metrics = {
        'Model': 'ResNet-18',
        'Training Loss': r.history['loss'][-1],
        'Validation Loss': r.history['val_loss'][-1],
        'Training Accuracy': r.history['accuracy'][-1],
        'Validation Accuracy': r.history['val_accuracy'][-1]
    }
    

In [ ]:
metrics_data.append(metrics)

In [ ]:
metrics_df = pd.DataFrame(metrics_data)

In [ ]:
metrics_df.to_csv('Metrics/Res-Net18model_metrics.csv', index=False)

In [ ]:
model.save_weights(r'training_1/mode_resnet18_weights.h5')
model.save(r'training_1/model_resnet18.h5')